# Milestone 2

Fix data preprocessing and observe effect on training with various data patterns.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt
import seaborn as sns
import re

In [28]:
# Read CSV Files
puppy_info = pd.read_excel('PuppyInfo.xls')
puppy_trainer_outcome = pd.read_excel('PuppyTrainerOutcome.xlsx')
trainer_info = pd.read_excel('TrainerInfo.xlsx')

print(len(puppy_info), len(puppy_trainer_outcome), len(trainer_info))

4782 28063 1688


In [29]:
puppy_info_rename = {
    'ogr_DogID': 'DogID',
    'Raiser_psn_PersonID': 'PersonID',
    'ExericeAmount': 'ExerciseAmount'
}
puppy_trainer_outcome_rename = {
    'dog_DogID': 'DogID',
    'ogr_PersonID': 'PersonID'
}

puppy_info.rename(columns=puppy_info_rename, inplace=True)
puppy_trainer_outcome.rename(columns=puppy_trainer_outcome_rename, inplace=True)

## Milestone 1 Code

Cleaned up and updated preprocessing code from milestone 1.

In [30]:
# Convert classes to cardinal values:
def col_classes_to_cardinal(df, col):
    col_set = df[col].unique()
    class_map = {}
    
    # Create mappings from classes to cardinal numbers
    for x in range(len(col_set)):
        class_map[col_set[x]] = x
        
    # Apply mapping
    return df[col].map(lambda x: class_map[x])

In [31]:
# Preprocess the sex column
def process_sex(df):
    # Mappings to proper label
    target_for_values = {
        'F': ['bitch','remale','f','Female','fem','fema;e','F','    F','Femae','Femail','Femaile','Femal','Femal3','Female','Femalw','femle','FEMALE','female','Girl','ID#2099','girl','n/a','None','own','Unknown','1364 & 655','1112/1329','065 102 601','2052','2235','11796','1972','1677','1649','1590','1395','1070','219','0','696','1018','ID# 2099','femal','femalw','Famale','femaile','femail'],
        'M': ['Male','1110','1231','1627','1644','1766','1870','2019','??','1JJ11','boy','Crate from Val and Jim Hazlin','don\'t have one.','M - neutered','maie','Mail','Maile','Make','make','Male - neutered','male (neutered)','"Male, neutered"','Male1832','mine doesn\'t have a number?','N/A','NA','Neutered Male','new crate','none','own crate','Weren\'t given a crate','m','male','MALE','Male', 'Male','neutered mail','mail','Male, neutered',' Neutered Male']
    }
    
    # Map to labels
    for k, v in target_for_values.items():
        df.loc[df.Sex.isin(v), 'Sex'] = k
    
    # Replace empty values with value for Male
    df['Sex'].fillna('M', inplace=True)

    # Convert classes to cardinal values
    df['Sex'] = puppy_info['Sex'].map({'F': 0, 'M': 1, 0: 0, 1: 1}).astype(int)

In [32]:
# Preprocess the attends_classes column
def process_attends_classes(df):
    # Replace missing values with the mean value 4.0
    df['AttendsClasses'].fillna(4.0, inplace=True)

In [33]:
# Preproces the exerice_amount column
def process_exercise_amount(df):
    # Method to get the first number found in a string
    def get_first_number(string):
        find_num = re.search(r'\d+', string)
        if(find_num == None):
            # Process words into numbers representing minutes
            if "one" in string or "an hour" in string:
                return 60
            elif "two" in string:
                return 120
            elif "three" in string:
                return 180
            elif "hour and half" in string:
                return 90
            elif "twenty" in string:
                return 20
            elif "forty" in string:
                return 45
            elif "hour" in string and not "hours" in string:
                return 60
            return 0

        # Process numbers into minute values
        num = int(find_num.group())
        if int(num < 10):
            return num*60
        return num
    
    # Method to group values together
    def averageTime(num, replaceZeroValue=60):
        if num == 0:
            return replaceZeroValue

        time_classes = [0, 20, 40, 60, 90, 120, 180, 440]
        for time_val in time_classes:
            if time_val >= num:
                return time_val

        return 440
    
    # Change to lowercase to regularize text and then map it
    puppy_exercise_amt = df["ExerciseAmount"].map(lambda x: str(x).lower())
    puppy_exercise_processed = list(map(lambda x: averageTime(int(get_first_number(str(x)))), puppy_exercise_amt))

    # Apply values to the column
    df["ExerciseAmount"] = puppy_exercise_processed

In [34]:
# Preprocess the breed column
def process_breed(df):
    # Method to process breed values
    def breed_mappings(breed):
        if breed == "golden labrador" or breed == "golden lab":
            return "golden labrador"
        elif breed in ["belgian shepherd", "belgen sharpart"]:
            return "belgian shepherd"
        elif "rador" in breed or "lab" in breed or breed[0:2] == "la" or breed in ["lr", "l", "bl", "labrador retriever"]:
            return "labrador retriever"
        elif "german shep" in breed or breed in ["gs", "gds", "greman shepherd", "geman shepherd", "gsd", "gsp", "g.s.", "german shepherd"] or "german sh" in breed or "shep" in breed:
            return "german shepherd"
        elif breed in ["gr", "golden retriever"] or "golden retr" in breed:
            return "golden retriever"
        elif breed in ["unknown", "black", "golden", "lym", "lbm", "ly m", "ged", "gdd", "noble", "nan"]:
            # Replace with the most common dog breed: Labrador retriever
            return "labrador retriever"
            # return "unknown"
        else:
            return breed
    
    # Map breed values and apply it to df
    df["Breed"] = df["Breed"].map(lambda x: breed_mappings(str(x).lower()))
    
    # Convert to cardinal values
    # df["Breed"] = col_classes_to_cardinal(df, "Breed")
    

In [35]:
# Preprocess the color column
def process_color(df):
    target_for_color = {
        'Sable': ['Coated Sable','Sable','sable'],
        'Yellow': ['blond','Blond/Yellow','Blonde','blondelab lode','Butterscotch','Carmel Yellow','cream','Cream','darkish brown','fox red','Gold','gold','golden','GOLDEN','Golden Yellow','Lab','light tan','light yellow','Light Yellow','red','Red','Red Fox','Rust','Tan','tan','WELLOW','Wheat','white','White','White and yellow','White/Yellow','Y','y','yel','Yel','Yellllow','Yello','YELLO','yello','yelloiw','Yellow','yellow','YELLOW','Yellow - Dark','Yellow (red)','Yellow & White','yellow lab','Yellow with black trim','Yellow/Butterscotch','yellow/cream','Yellow/White','yellow1','yellowf','Light yellow','Yellowf'],
        'Golden': ['camel','golden/red','goldish','honey','Light Golden','Medium Gold','red/gold','reddish gold','warm gold','warm honey','Tan/Gold'],
        'Black/tan': ['B & T','b/t','B&T','B+T','bl and tan','Black & Tan','Black &tan','Black + Tan','Black and ran','Black and tan','Black and tan (?)','Black Brown','black tan','black w/ tan','Black, tan','Black, tan, silver','Black,tan','Black/ Tan','black/brown','Black/Tan','black+ tan','Blk & Tan','Blk and Tan','Blk/Tan','Brown & Black','brown black','Brown-Black','Brown, black','Brown/Black','Brown/Black/Tan','Coated Black','tan and black','Tan/Black','black and tan','Black and Tan','Bicolor (Black & red)','Bicolor (black w/ brown legs)','Black & red','Black and ran','Black and Red','black and white','Tri','Tri color','Brindle','GSD','B/T','b&t','Black / Tan','black & tan','black, tan','Black/tan','blk and tan','Blk/TAn','tan/black','Blk and tan','Black & tan','black/tan',],
        'Black': ['Color', 'B','Bl','Bl','blac','black (and beautiful)','blck,','Blk','Blk.','blsck','back','black','BLACK','blk','BLK','color','lab',' Black','blck','Back','BLK.','BLack','nan']
    }
    
    # Replace messy color values
    for k, v in target_for_color.items():
        df.loc[df.Color.isin(v), 'Color'] = k
    
    # Replaced missing values with Black
    df['Color'].fillna('Black',inplace = True)
    
    # # Conver to cardinal values
    # df['Color'] = col_classes_to_cardinal(df, 'Color')

In [36]:
# Preprocess the status column
def process_status(df):    
    df['dog_SubStatusCode'].replace([23,25,26,27,55,98,121,169],[1,1,1,1,1,1,1,1], inplace=True)
    df['dog_SubStatusCode'][df.dog_SubStatusCode != 1] = 0

process_status(puppy_trainer_outcome)

/Users/williamnguyen/anaconda3/envs/cs274/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [37]:
# Perform all preprocessing steps

process_sex(puppy_info)
print(puppy_info['Sex'].value_counts())

process_attends_classes(puppy_info)

process_exercise_amount(puppy_info)
print(puppy_info['ExerciseAmount'].value_counts())

process_breed(puppy_info)
print(puppy_info['Breed'].value_counts())

process_color(puppy_info)
print(puppy_info['Color'].value_counts())

1    2400
0    2382
Name: Sex, dtype: int64
60     2137
120     924
90      608
40      604
180     263
20      147
440      99
Name: ExerciseAmount, dtype: int64
labrador retriever    4397
german shepherd        347
golden retriever        33
golden labrador          4
belgian shepherd         1
Name: Breed, dtype: int64
Yellow       2281
Black        2087
Black/tan     348
Sable          40
Golden         26
Name: Color, dtype: int64


## Part 1
 Based on the feedback in your milestone 1, prepare your data again that would meet the better prediction goal. Explain any steps you do and how it will help. Prepare 5 or more sets of data with varying number of features. 10 features that you selected in Milestone 1 should be one of these 5 cases.

In [73]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

OHE_columns = {}

# Encode Values into a new column
def column_encode(df, col_name):
    lb_make = LabelEncoder()
    df[col_name + "_encoded"] = lb_make.fit_transform(df[col_name])

# One Hot Encoder
def one_hot_encode(df, col_name):
    lb_style = LabelBinarizer()
    lb_results = lb_style.fit_transform(df[col_name])
    new_df = pd.DataFrame(lb_results, columns=lb_style.classes_)
    
    # Add new_df to df
    print("New columns added:")
    for x in new_df:
        new_col_name = str(col_name) + "-" + str(x)
        if isinstance(x, np.float64):
            new_col_name = str(col_name) + "-" + str(int(x))
            
        df[new_col_name] = new_df[x]
        print(" * " + new_col_name)
        
        if col_name not in OHE_columns:
            OHE_columns[col_name] = []
        OHE_columns[col_name].append(new_col_name)

In [74]:
# Apply One-Hot Encoding to Certain Columns
# 
columns_to_OHE = [
    "Breed",
    "AttendsClasses",
    "AttendsHomeSwitches",
    "ExerciseAmount",
    "Housemanners",
    "FriendlyWAnimals",
    "JumpOnPeople",
    "StaysOnCommand",
    "TrafficFear",
    "Color",
    "Health",
    "EnergyLevel",
    "StealsFood",
    "BehavesWellClass",
    "RaidsGarbage",
    "Stairs"
]
for col in columns_to_OHE:
    column_encode(puppy_info, col)
    one_hot_encode(puppy_info, col)

New columns added:
 * Breed-belgian shepherd
 * Breed-german shepherd
 * Breed-golden labrador
 * Breed-golden retriever
 * Breed-labrador retriever
New columns added:
 * AttendsClasses-0
 * AttendsClasses-1
 * AttendsClasses-2
 * AttendsClasses-3
 * AttendsClasses-4
 * AttendsClasses-5
New columns added:
 * AttendsHomeSwitches-0
 * AttendsHomeSwitches-1
 * AttendsHomeSwitches-2
 * AttendsHomeSwitches-3
 * AttendsHomeSwitches-4
 * AttendsHomeSwitches-5
New columns added:
 * ExerciseAmount-20
 * ExerciseAmount-40
 * ExerciseAmount-60
 * ExerciseAmount-90
 * ExerciseAmount-120
 * ExerciseAmount-180
 * ExerciseAmount-440
New columns added:
 * Housemanners-0
 * Housemanners-1
 * Housemanners-2
 * Housemanners-3
 * Housemanners-4
 * Housemanners-5
New columns added:
 * FriendlyWAnimals-0
 * FriendlyWAnimals-1
 * FriendlyWAnimals-2
 * FriendlyWAnimals-3
 * FriendlyWAnimals-4
 * FriendlyWAnimals-5
New columns added:
 * JumpOnPeople-0
 * JumpOnPeople-1
 * JumpOnPeople-2
 * JumpOnPeople-3
 * Ju

### Merge Data

In [75]:
puppy_info = puppy_info.drop_duplicates(['DogID'], keep='first')
puppy_trainer_outcome = puppy_trainer_outcome.drop_duplicates(['DogID'], keep='first')

print(len(puppy_info), len(puppy_trainer_outcome))

2138 12677


In [76]:
puppy_merged = puppy_info.merge(puppy_trainer_outcome, on=['DogID'], how='inner')

print("%s rows in puppy_info" % len(puppy_info))
print("%s rows in puppy_trainer_outcome" % len(puppy_trainer_outcome))
print("%s rows in merged data" % len(puppy_merged))

2138 rows in puppy_info
12677 rows in puppy_trainer_outcome
2110 rows in merged data


In [77]:
desired_columns = [
    'StealsFood',
    'EnergyLevel',
    'JumpOnPeople',
    'AttendsClasses',
    'FriendlyWAnimals',
    'Health',
    'AttendsHomeSwitches',
    'TrafficFear',
    'Sex',
    'StaysOnCommand',
    'RaidsGarbage',
    'BehavesWellClass',
    'Stairs',
    'Breed',
    'Color',
    'ExerciseAmount',
    'Housemanners',
    'dog_SubStatusCode'
]

desired_columns_processed = []

# Only keep OHE columns, not the original columns
for col in desired_columns:
    if col in OHE_columns:
        for OHE_col in OHE_columns[col]:
            desired_columns_processed.append(OHE_col)
    else:
        desired_columns_processed.append(col)
        
# Write all columns to 

processed_data = puppy_merged[desired_columns_processed]

### Write Data to CSV

In [78]:
processed_data.to_csv('ProcessedPuppyInfo.csv', index=False)

## Part 2
For your selected ML algorithm (group 1:Naive Bayes , group 2:Decision Trees , group 3: Linear/Logic Regression, group 4:SVM)

a. Test out on your Spark MLLib algorithm on each set of data prepared in (a). Observe the prediction outcome based on puppy info alone and offer explanation.

b. Repeat 2.a few times. Do you see consistent prediction for the same set of data when it is randomly split? How about when the data is not randomly split?

c. What is the optimal number of features in your datasets from (1) that offers best prediction rate?

d. What parameters will in your algorithm will affect the prediction rate? How and why?

In [83]:
# Column Mappings for Feature Selections
feature_set_columns = [
    ["dog_SubStatusCode", 'StealsFood', 'EnergyLevel', 'JumpOnPeople', 'AttendsClasses', 'FriendlyWAnimals', 'Health', 'AttendsHomeSwitches', 'TrafficFear', 'Sex', 'StaysOnCommand', 'RaidsGarbage', 'BehavesWellClass', 'Stairs', 'Breed', 'Color', 'ExerciseAmount', 'Housemanners'],
    ["dog_SubStatusCode", "AttendsClasses", "AttendsHomeSwitches", "ExerciseAmount", "Breed", "Sex", "Housemanners", "FriendlyWAnimals", "JumpOnPeople", "StaysOnCommand", "TrafficFear"],
    ["dog_SubStatusCode", "AttendsClasses", "Housemanners", "StaysOnCommand", "Health", "EnergyLevel"],
    ["dog_SubStatusCode", "Sex", "Color", "FriendlyWAnimals", "ExerciseAmount", "RaidsGarbage"],
    ["dog_SubStatusCode", "StaysOnCommand", "Health", "FriendlyWAnimals"],
    ["dog_SubStatusCode", "Color", "Health", "EnergyLevel", "StealsFood", "BehavesWellClass", "RaidsGarbage", "Stairs"]
]

mapped_feature_set_columns = []

for some_set in feature_set_columns:
    new_set = []
    for col in some_set:
        if col in OHE_columns:
            for OHE_col in OHE_columns[col]:
                new_set.append(OHE_col)
        else:
            new_set.append(col)
    mapped_feature_set_columns.append(new_set)
    
for some_set in mapped_feature_set_columns:
    set_str = ""
    for col in some_set:
        set_str += "\"" + col + "\", "
    
    print(set_str)
    print()

"dog_SubStatusCode", "StealsFood-0", "StealsFood-1", "StealsFood-2", "StealsFood-3", "StealsFood-4", "StealsFood-5", "EnergyLevel-0", "EnergyLevel-1", "EnergyLevel-2", "EnergyLevel-3", "EnergyLevel-4", "EnergyLevel-5", "JumpOnPeople-0", "JumpOnPeople-1", "JumpOnPeople-2", "JumpOnPeople-3", "JumpOnPeople-4", "JumpOnPeople-5", "AttendsClasses-0", "AttendsClasses-1", "AttendsClasses-2", "AttendsClasses-3", "AttendsClasses-4", "AttendsClasses-5", "FriendlyWAnimals-0", "FriendlyWAnimals-1", "FriendlyWAnimals-2", "FriendlyWAnimals-3", "FriendlyWAnimals-4", "FriendlyWAnimals-5", "Health-0", "Health-1", "Health-2", "Health-3", "Health-4", "Health-5", "AttendsHomeSwitches-0", "AttendsHomeSwitches-1", "AttendsHomeSwitches-2", "AttendsHomeSwitches-3", "AttendsHomeSwitches-4", "AttendsHomeSwitches-5", "TrafficFear-0", "TrafficFear-1", "TrafficFear-2", "TrafficFear-3", "TrafficFear-4", "TrafficFear-5", "Sex", "StaysOnCommand-0", "StaysOnCommand-1", "StaysOnCommand-2", "StaysOnCommand-3", "StaysOnCo

In [84]:
# Output CSVs for each Feature Set
set_names = ['A', 'B', 'C', 'D', 'E', 'F']

for i in range(len(mapped_feature_set_columns)):
    feature_set = processed_data[mapped_feature_set_columns[i]]
    feature_set.to_csv('puppy_info_feature_set_' + set_names[i] + '.csv', index=False)

In [ ]:
/FileStore/tables/qb42coi11494469601318/puppy_info_feature_set_A.csv
/FileStore/tables/gslsth1d1494469669982/puppy_info_feature_set_B.csv
/FileStore/tables/eto03ixw1494469788598/puppy_info_feature_set_C.csv
/FileStore/tables/3x5kh0e31494469817043/puppy_info_feature_set_D.csv
/FileStore/tables/vf2nxmj91494469852583/puppy_info_feature_set_E.csv
/FileStore/tables/yhcafpdn1494469880969/puppy_info_feature_set_F.csv


## Part 3 
Normalization and Feature Extraction of text data

Many columns are in verbal text form. One way to make use of these data for learning and prediction purpose is to normalize the given text data and then perform feature extraction. Many methodologies can be applied for machine learning purposes after this step.

a. For DayInLife column in TrainerInfo.xslt, normalize all the data within. See text normalization definition in Wiki (Links to an external site.)Links to an external site.

b. Perform feature extraction suitable for our prediction need. Explain what feature extraction in Spark MLlib (Links to an external site.)Links to an external site. are useful for this purpose. Show one or more such feature extraction.

c. What would you propose to do next after 3.b for our prediction need?